# Imports

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [2]:
# export
from exp.nb_12 import *

/Users/imad/anaconda3/envs/dl/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Data

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=6317)

In [4]:
path = untar_data(URLs.IMDB)

We have to preprocess the data again to pickle it because if we try to load the previous `SplitLabeledData` with pickle, it will complain some of the functions aren't in main.

In [5]:
il = TextList.from_files(path, include=['train', 'test', 'unsup'])
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [6]:
proc_tok,proc_num = TokenizeProcessor(max_workers=8),NumericalizeProcessor()

In [7]:
ll = label_by_func(sd, lambda x: 0, proc_x = [proc_tok,proc_num])

In [10]:
pickle.dump(ll, open(path / "ll_lm.pkl", "wb"))
pickle.dump(proc_num.vocab, open(path / "vocab_lm.pkl", "wb"))

In [11]:
ll = pickle.load(open(path / "ll_lm.pkl", "rb"))
vocab = pickle.load(open(path / "vocab_lm.pkl", "rb"))

In [12]:
bs, bptt = 64, 70
data = lm_databunchify(ll, bs, bptt)

torch.Size([26980499])
torch.Size([64, 421570])
torch.Size([3003258])
torch.Size([64, 46925])


# AWD-LSTM

Before explaining what an AWD LSTM is, we need to start with an LSTM. RNNs were covered in part 1, if you need a refresher, there is a great visualization of them on [this website](http://joshvarty.github.io/VisualizingRNNs/).

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=6330)

### LSTM from scratch

We need to implement those equations (where $\sigma$ stands for sigmoid):

![LSTM cell and equations](images/lstm.jpg)
(picture from [Understanding LSTMs](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) by Chris Olah.)

If we want to take advantage of our GPU, it's better to do one big matrix multiplication than four smaller ones. So we compute the values of the four gates all at once. Since there is a matrix multiplication and a bias, we use `nn.Linear` to do it.

We need two linear layers: one for the input and one for the hidden state.

In [8]:
input_sz = 10
hidden_sz = 20
n_layers = 2
bs = 3
seq_len = 5
rnn = nn.LSTM(input_sz, hidden_sz, n_layers, batch_first=True)

In [9]:
# W_ih: 4 x hidden_sz, input_sz for first layer then 4 x hidden_sz, hidden_sz
# W_hh: 4 x hidden_sz, hidden_sz
# b_ih: 4 x hidden_sz
# b_hh: 4 x hidden_sz
[param.shape for param in rnn.parameters()]

[torch.Size([80, 10]),
 torch.Size([80, 20]),
 torch.Size([80]),
 torch.Size([80]),
 torch.Size([80, 20]),
 torch.Size([80, 20]),
 torch.Size([80]),
 torch.Size([80])]

In [10]:
# h_0 and c_0: n_layers x bs x hidden_sz
h0 = torch.randn(n_layers, bs, hidden_sz)
c0 = torch.randn(n_layers, bs, hidden_sz)
output, (hn, cn) = rnn(torch.randn(bs, seq_len, input_sz), (h0, c0))

In [11]:
output.shape

torch.Size([3, 5, 20])

In [12]:
h0.shape, c0.shape

(torch.Size([2, 3, 20]), torch.Size([2, 3, 20]))

In [41]:
class LSTMCell(nn.Module):
    def __init__(self, ni, nh):
        super().__init__()
        self.ih = nn.Linear(ni, 4 * nh)
        self.hh = nn.Linear(nh, 4 * nh)

    def forward(self, input, state):
        h, c = state
        # One big multiplication for all the gates is better than 4 smaller ones
        # a = self.ih(input)
        # b = self.hh(h)
        # print(a.shape)
        # print(b.shape)
        gates = (self.ih(input) + self.hh(h)).chunk(4, 1)
        ingate, forgetgate, outgate = map(torch.sigmoid, gates[:3])
        cellgate = gates[3].tanh()

        c = (forgetgate * c) + (ingate * cellgate)
        h = outgate * c.tanh()
        return h, (h, c)

Then an LSTM layer just applies the cell on all the time steps in order.

In [42]:
class LSTMLayer(nn.Module):
    def __init__(self, cell, *cell_args):
        super().__init__()
        self.cell = cell(*cell_args)

    def forward(self, input, state):
        # print(input.shape)
        inputs = input.unbind(1)
        # print(inputs[0].shape)
        outputs = []
        for i in range(len(inputs)):
            out, state = self.cell(inputs[i], state)
            outputs += [out]
        return torch.stack(outputs, dim=1), state

Now let's try it out and see how fast we are. We only measure the forward pass.

In [43]:
lstm = LSTMLayer(LSTMCell, 300, 300)

In [44]:
x = torch.randn(64, 5, 300)
h = (torch.zeros(64, 300), torch.zeros(64, 300))

In [48]:
y, state = lstm(x, h)
y.shape, state[0].shape, state[1].shape

(torch.Size([64, 5, 300]), torch.Size([64, 300]), torch.Size([64, 300]))

CPU

In [49]:
%timeit -n 10 y,h1 = lstm(x,h)

10.8 ms ± 5.99 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
lstm = lstm.cuda()
x = x.cuda()
h = (h[0].cuda(), h[1].cuda())

In [ ]:
def time_fn(f):
    f()
    torch.cuda.synchronize()

CUDA

In [ ]:
f = partial(lstm,x,h)
time_fn(f)

In [ ]:
%timeit -n 10 time_fn(f)

### Builtin version

Let's compare with PyTorch!

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=6718)

In [ ]:
lstm = nn.LSTM(300, 300, 1, batch_first=True)

In [ ]:
x = torch.randn(64, 70, 300)
h = (torch.zeros(1, 64, 300),torch.zeros(1, 64, 300))

CPU

In [ ]:
%timeit -n 10 y,h1 = lstm(x,h)

In [ ]:
lstm = lstm.cuda()
x = x.cuda()
h = (h[0].cuda(), h[1].cuda())

In [ ]:
f = partial(lstm,x,h)
time_fn(f)

GPU

In [ ]:
%timeit -n 10 time_fn(f)

So our version is running at almost the same speed on the CPU. However, on the GPU, PyTorch uses CuDNN behind the scenes that optimizes greatly the for loop.

### Jit version

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=6744)

In [20]:
import torch.jit as jit
from torch import Tensor

We have to write everything from scratch to be a bit faster, so we don't use the linear layers here.

In [21]:
class LSTMCell(jit.ScriptModule):
    def __init__(self, ni, nh):
        super().__init__()
        self.ni = ni
        self.nh = nh
        self.w_ih = nn.Parameter(torch.randn(4 * nh, ni))
        self.w_hh = nn.Parameter(torch.randn(4 * nh, nh))
        self.bias_ih = nn.Parameter(torch.randn(4 * nh))
        self.bias_hh = nn.Parameter(torch.randn(4 * nh))

    @jit.script_method
    def forward(
        self, input: Tensor, state: Tuple[Tensor, Tensor]
    ) -> Tuple[Tensor, Tuple[Tensor, Tensor]]:
        hx, cx = state
        gates = input @ self.w_ih.t() + self.bias_ih + hx @ self.w_hh.t() + self.bias_hh
        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)

        ingate = torch.sigmoid(ingate)
        forgetgate = torch.sigmoid(forgetgate)
        cellgate = torch.tanh(cellgate)
        outgate = torch.sigmoid(outgate)

        cy = (forgetgate * cx) + (ingate * cellgate)
        hy = outgate * torch.tanh(cy)

        return hy, (hy, cy)

In [22]:
class LSTMLayer(jit.ScriptModule):
    def __init__(self, cell, *cell_args):
        super().__init__()
        self.cell = cell(*cell_args)

    @jit.script_method
    def forward(
        self, input: Tensor, state: Tuple[Tensor, Tensor]
    ) -> Tuple[Tensor, Tuple[Tensor, Tensor]]:
        inputs = input.unbind(1)
        outputs = []
        for i in range(len(inputs)):
            out, state = self.cell(inputs[i], state)
            outputs += [out]
        return torch.stack(outputs, dim=1), state

In [ ]:
lstm = LSTMLayer(LSTMCell, 300, 300)

In [ ]:
x = torch.randn(64, 70, 300)
h = (torch.zeros(64, 300),torch.zeros(64, 300))

In [ ]:
%timeit -n 10 y,h1 = lstm(x,h)

In [ ]:
lstm = lstm.cuda()
x = x.cuda()
h = (h[0].cuda(), h[1].cuda())

In [ ]:
f = partial(lstm,x,h)
time_fn(f)

In [ ]:
%timeit -n 10 time_fn(f)

With jit, we almost get to the CuDNN speed!

### Dropout

We want to use the AWD-LSTM from [Stephen Merity et al.](https://arxiv.org/abs/1708.02182). First, we'll need all different kinds of dropouts. Dropout consists into replacing some coefficients by 0 with probability p. To ensure that the average of the weights remains constant, we apply a correction to the weights that aren't nullified of a factor `1/(1-p)` (think of what happens to the activations if you want to figure out why!)

We usually apply dropout by drawing a mask that tells us which elements to nullify or not:

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=7003)

In [23]:
# export
def dropout_mask(x, sz, p):
    return x.new(*sz).bernoulli_(1 - p).div_(1 - p)

In [24]:
x = torch.randn(10, 10)
mask = dropout_mask(x, (10, 10), 0.5)
mask

tensor([[2., 0., 2., 0., 0., 2., 2., 0., 0., 2.],
        [2., 0., 0., 2., 0., 2., 2., 2., 2., 2.],
        [2., 0., 0., 0., 2., 2., 2., 2., 0., 2.],
        [0., 0., 0., 0., 2., 0., 2., 0., 2., 2.],
        [0., 0., 2., 2., 0., 0., 0., 2., 2., 0.],
        [2., 2., 2., 2., 2., 0., 0., 0., 2., 0.],
        [2., 0., 2., 2., 2., 2., 0., 0., 0., 2.],
        [0., 0., 2., 0., 0., 0., 2., 0., 0., 2.],
        [2., 2., 0., 2., 2., 0., 2., 2., 2., 2.],
        [0., 2., 0., 2., 2., 2., 2., 2., 0., 2.]])

Once with have a dropout mask `mask`, applying the dropout to `x` is simply done by `x = x * mask`. We create our own dropout mask and don't rely on pytorch dropout because we do not want to nullify all the coefficients randomly: on the sequence dimension, we will want to have always replace the same positions by zero along the seq_len dimension.

In [25]:
(x * mask).std(), x.std()

(tensor(1.5708), tensor(1.0497))

Inside a RNN, a tensor x will have three dimensions: __bs x seq_len x vocab_size__.  Recall that we want to consistently apply the dropout mask across the seq_len dimension, therefore, we create a dropout mask for the first and third dimension and broadcast it to the seq_len dimension.

In [26]:
# export
class RNNDropout(nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.p = p

    def forward(self, x):
        if not self.training or self.p == 0.0:
            return x
        m = dropout_mask(x.data, (x.size(0), 1, x.size(2)), self.p)
        return x * m

In [27]:
dp = RNNDropout(0.3)
tst_input = torch.randn(3, 3, 7)
tst_input, dp(tst_input)

(tensor([[[ 0.1125,  0.7893, -0.7071,  0.3714,  0.2993, -0.7827,  0.2123],
          [ 1.6228,  0.2041,  0.4360, -0.4757,  0.0936, -2.1518,  0.8498],
          [-0.2057, -1.0406, -1.0485, -0.4324, -0.4264,  0.6183, -1.5476]],
 
         [[ 0.4590, -1.0464, -1.7392, -0.9459, -0.5021,  1.0539, -0.2184],
          [-2.1797, -0.6710, -3.1562, -1.7733, -0.7304, -0.3632, -0.4986],
          [-0.4415,  1.2148,  0.5129, -0.6251, -0.4439, -0.5832, -0.2418]],
 
         [[ 0.4578, -1.4622, -1.7672, -0.4888,  0.0287,  0.1496,  0.8046],
          [-1.1317,  0.5457,  2.4901, -0.9452,  0.4740,  1.6722, -0.5892],
          [-1.3342,  1.3715,  1.2401,  1.0104,  0.6824, -0.1213, -0.2246]]]),
 tensor([[[ 0.1607,  0.0000, -1.0101,  0.5306,  0.4276, -1.1181,  0.0000],
          [ 2.3182,  0.0000,  0.6228, -0.6796,  0.1337, -3.0740,  0.0000],
          [-0.2939, -0.0000, -1.4978, -0.6178, -0.6092,  0.8832, -0.0000]],
 
         [[ 0.6557, -1.4949, -2.4845, -1.3513, -0.7173,  1.5056, -0.0000],
          [-3

WeightDropout is dropout applied to the weights of the inner LSTM hidden to hidden matrix. This is a little hacky if we want to preserve the CuDNN speed and not reimplement the cell from scratch. We add a parameter that will contain the raw weights, and we replace the weight matrix in the LSTM at the beginning of the forward pass.

In [28]:
# export
import warnings

WEIGHT_HH = "weight_hh_l0"


class WeightDropout(nn.Module):
    def __init__(self, module, weight_p=[0.0], layer_names=[WEIGHT_HH]):
        super().__init__()
        self.module, self.weight_p, self.layer_names = module, weight_p, layer_names
        for layer in self.layer_names:
            # Makes a copy of the weights of the selected layers.
            w = getattr(self.module, layer)
            self.register_parameter(f"{layer}_raw", nn.Parameter(w.data))
            self.module._parameters[layer] = F.dropout(
                w, p=self.weight_p, training=False
            )

    def _setweights(self):
        for layer in self.layer_names:
            raw_w = getattr(self, f"{layer}_raw")
            self.module._parameters[layer] = F.dropout(
                raw_w, p=self.weight_p, training=self.training
            )

    def forward(self, *args):
        self._setweights()
        with warnings.catch_warnings():
            # To avoid the warning that comes because the weights aren't flattened.
            warnings.simplefilter("ignore")
            return self.module.forward(*args)

Let's try it!

In [29]:
module = nn.LSTM(5, 2)
dp_module = WeightDropout(module, 0.4)
getattr(dp_module.module, WEIGHT_HH)

Parameter containing:
tensor([[ 0.3713,  0.4039],
        [ 0.1371, -0.6039],
        [-0.3796, -0.2821],
        [-0.6853, -0.3837],
        [ 0.5093,  0.3696],
        [ 0.2082,  0.3911],
        [ 0.4724, -0.5715],
        [-0.3039,  0.4216]], requires_grad=True)

It's at the beginning of a forward pass that the dropout is applied to the weights.

In [30]:
tst_input = torch.randn(4, 20, 5)
h = (torch.zeros(1, 20, 2), torch.zeros(1, 20, 2))
x, h = dp_module(tst_input, h)
getattr(dp_module.module, WEIGHT_HH)

tensor([[ 0.6189,  0.0000],
        [ 0.2285, -1.0065],
        [-0.0000, -0.4702],
        [-1.1421, -0.0000],
        [ 0.0000,  0.0000],
        [ 0.3471,  0.6518],
        [ 0.7873, -0.9525],
        [-0.5065,  0.7027]], grad_fn=<MulBackward0>)

EmbeddingDropout applies dropout to full rows of the embedding matrix.

In [31]:
# export
class EmbeddingDropout(nn.Module):
    "Applies dropout in the embedding layer by zeroing out some elements of the embedding vector."

    def __init__(self, emb, embed_p):
        super().__init__()
        self.emb, self.embed_p = emb, embed_p
        self.pad_idx = self.emb.padding_idx
        if self.pad_idx is None:
            self.pad_idx = -1

    def forward(self, words, scale=None):
        if self.training and self.embed_p != 0:
            size = (self.emb.weight.size(0), 1)
            mask = dropout_mask(self.emb.weight.data, size, self.embed_p)
            masked_embed = self.emb.weight * mask
        else:
            masked_embed = self.emb.weight
        if scale:
            masked_embed.mul_(scale)
        return F.embedding(
            words,
            masked_embed,
            self.pad_idx,
            self.emb.max_norm,
            self.emb.norm_type,
            self.emb.scale_grad_by_freq,
            self.emb.sparse,
        )

In [ ]:
enc = nn.Embedding(100, 7, padding_idx=1)
enc_dp = EmbeddingDropout(enc, 0.5)
tst_input = torch.randint(0,100,(8,))
enc_dp(tst_input)

In [69]:
m = nn.Sequential(nn.Linear(2, 2))

In [73]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 10)
        self.lstm = nn.LSTM(10, 20, 3)
        self.seq = nn.Sequential(nn.Linear(2, 3), nn.ReLU())

    def forward(self, x):
        pass

In [77]:
m = Net()
m

Net(
  (linear): Linear(in_features=2, out_features=10, bias=True)
  (lstm): LSTM(10, 20, num_layers=3)
  (seq): Sequential(
    (0): Linear(in_features=2, out_features=3, bias=True)
    (1): ReLU()
  )
)

In [76]:
list(m.children())

[Linear(in_features=2, out_features=10, bias=True),
 LSTM(10, 20, num_layers=3),
 Sequential(
   (0): Linear(in_features=2, out_features=3, bias=True)
   (1): ReLU()
 )]

In [82]:
list(m.named_modules())

[('',
  Net(
    (linear): Linear(in_features=2, out_features=10, bias=True)
    (lstm): LSTM(10, 20, num_layers=3)
    (seq): Sequential(
      (0): Linear(in_features=2, out_features=3, bias=True)
      (1): ReLU()
    )
  )),
 ('linear', Linear(in_features=2, out_features=10, bias=True)),
 ('lstm', LSTM(10, 20, num_layers=3)),
 ('seq',
  Sequential(
    (0): Linear(in_features=2, out_features=3, bias=True)
    (1): ReLU()
  )),
 ('seq.0', Linear(in_features=2, out_features=3, bias=True)),
 ('seq.1', ReLU())]

In [50]:
# list(m.named_parameters())

In [87]:
getattr(m.linear, "weight").data

tensor([[ 0.6062,  0.6448],
        [-0.3371, -0.4139],
        [-0.3318,  0.6996],
        [-0.6662, -0.4964],
        [-0.3403,  0.1020],
        [ 0.3867,  0.4497],
        [-0.5221,  0.4778],
        [ 0.5745, -0.2707],
        [-0.6860,  0.6585],
        [-0.5408,  0.4873]])

In [72]:
[attr for attr in dir(m) if not attr.startswith("__")]

['T_destination',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_item_by_idx',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_version',
 'add_module',
 'apply',
 'bfloat16',
 'buffers',
 'children',
 'cpu',
 'cuda',
 'double',
 'dump_patches',
 'eval',
 'extra_repr',
 'float',
 'forward',
 'get_buffer',
 'get_parameter',
 'get_submodule',
 'half',
 'load_state_dict',
 'modules',
 'named_buffers',
 'named_children',
 'named_modules',
 'named_parameters',
 'parameters',
 'register_backward_hook',
 'register_buffer',
 'register_forward_hook',
 'register_forward_pre_hook',


### Main model

The main model is a regular LSTM with several layers, but using all those kinds of dropouts.

In [32]:
# export
def to_detach(h):
    "Detaches `h` from its history."
    return h.detach() if type(h) == torch.Tensor else tuple(to_detach(v) for v in h)

In [33]:
# export
class AWD_LSTM(nn.Module):
    "AWD-LSTM inspired by https://arxiv.org/abs/1708.02182."
    initrange = 0.1

    def __init__(
        self,
        vocab_sz,
        emb_sz,
        n_hid,
        n_layers,
        pad_token,
        hidden_p=0.2,
        input_p=0.6,
        embed_p=0.1,
        weight_p=0.5,
    ):
        super().__init__()
        self.bs, self.emb_sz, self.n_hid, self.n_layers = 1, emb_sz, n_hid, n_layers
        self.emb = nn.Embedding(vocab_sz, emb_sz, padding_idx=pad_token)
        self.emb_dp = EmbeddingDropout(self.emb, embed_p)
        self.rnns = [
            nn.LSTM(
                emb_sz if l == 0 else n_hid,
                (n_hid if l != n_layers - 1 else emb_sz),
                1,
                batch_first=True,
            )
            for l in range(n_layers)
        ]
        self.rnns = nn.ModuleList([WeightDropout(rnn, weight_p) for rnn in self.rnns])
        self.emb.weight.data.uniform_(-self.initrange, self.initrange)
        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(n_layers)])

    def forward(self, input):
        bs, sl = input.size()
        if bs != self.bs:
            self.bs = bs
            self.reset()
        raw_output = self.input_dp(self.emb_dp(input))
        new_hidden, raw_outputs, outputs = [], [], []
        for l, (rnn, hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
            raw_output, new_h = rnn(raw_output, self.hidden[l])
            new_hidden.append(new_h)
            raw_outputs.append(raw_output)
            if l != self.n_layers - 1:
                raw_output = hid_dp(raw_output)
            outputs.append(raw_output)
        self.hidden = to_detach(new_hidden)
        return raw_outputs, outputs

    def _one_hidden(self, l):
        "Return one hidden state."
        nh = self.n_hid if l != self.n_layers - 1 else self.emb_sz
        return next(self.parameters()).new(1, self.bs, nh).zero_()

    def reset(self):
        "Reset the hidden states."
        self.hidden = [
            (self._one_hidden(l), self._one_hidden(l)) for l in range(self.n_layers)
        ]

On top of this, we will apply a linear decoder. It's often best to use the same matrix as the one for the embeddings in the weights of the decoder.

In [34]:
# export
class LinearDecoder(nn.Module):
    def __init__(self, n_out, n_hid, output_p, tie_encoder=None, bias=True):
        super().__init__()
        self.output_dp = RNNDropout(output_p)
        self.decoder = nn.Linear(n_hid, n_out, bias=bias)
        if bias:
            self.decoder.bias.data.zero_()
        if tie_encoder:
            self.decoder.weight = tie_encoder.weight
        else:
            init.kaiming_uniform_(self.decoder.weight)

    def forward(self, input):
        raw_outputs, outputs = input
        output = self.output_dp(outputs[-1]).contiguous()
        decoded = self.decoder(
            output.view(output.size(0) * output.size(1), output.size(2))
        )
        return decoded, raw_outputs, outputs

In [35]:
# export
class SequentialRNN(nn.Sequential):
    "A sequential module that passes the reset call to its children."

    def reset(self):
        for c in self.children():
            if hasattr(c, "reset"):
                c.reset()

And now we stack all of this together!

In [36]:
# export
def get_language_model(
    vocab_sz,
    emb_sz,
    n_hid,
    n_layers,
    pad_token,
    output_p=0.4,
    hidden_p=0.2,
    input_p=0.6,
    embed_p=0.1,
    weight_p=0.5,
    tie_weights=True,
    bias=True,
):
    rnn_enc = AWD_LSTM(
        vocab_sz,
        emb_sz,
        n_hid=n_hid,
        n_layers=n_layers,
        pad_token=pad_token,
        hidden_p=hidden_p,
        input_p=input_p,
        embed_p=embed_p,
        weight_p=weight_p,
    )
    enc = rnn_enc.emb if tie_weights else None
    return SequentialRNN(
        rnn_enc, LinearDecoder(vocab_sz, emb_sz, output_p, tie_encoder=enc, bias=bias)
    )

In [37]:
tok_pad = vocab.index(PAD)

Now we can test this all works without throwing a bug.

In [38]:
tst_model = get_language_model(len(vocab), 300, 300, 2, tok_pad)
tst_model = tst_model.cuda()

AssertionError: Torch not compiled with CUDA enabled

In [39]:
x, y = next(iter(data.train_dl))

In [40]:
z = tst_model(x.cuda())

AssertionError: Torch not compiled with CUDA enabled

We return three things to help with regularization: the true output (probabilities for each word), but also the activations of the encoder, with or without dropouts.

In [41]:
len(z)

NameError: name 'z' is not defined

In [42]:
decoded, raw_outputs, outputs = z

NameError: name 'z' is not defined

The decoded tensor is flattened to `bs * seq_len` by `len(vocab)`:

In [ ]:
decoded.size()

`raw_outputs` and `outputs` each contain the results of the intermediary layers:

In [ ]:
len(raw_outputs),len(outputs)

In [ ]:
[o.size() for o in raw_outputs], [o.size() for o in outputs]

### Callbacks to train the model

We need to add a few tweaks to train a language model: first we will clip the gradients. This is a classic technique that will allow us to use a higher learning rate by putting a maximum value on the norm of the gradients.

In [43]:
# export
class GradientClipping(Callback):
    def __init__(self, clip=None):
        self.clip = clip

    def after_backward(self):
        if self.clip:
            nn.utils.clip_grad_norm_(self.run.model.parameters(), self.clip)

Then we add an `RNNTrainer` that will do four things:
- change the output to make it contain only the `decoded` tensor (for the loss function) and store the `raw_outputs` and `outputs`
- apply Activation Regularization (AR): we add to the loss an L2 penalty on the last activations of the AWD LSTM (with dropout applied)
- apply Temporal Activation Regularization (TAR): we add to the loss an L2 penalty on the difference between two consecutive (in terms of words) raw outputs
- trigger the shuffle of the LMDataset at the beginning of each epoch

In [44]:
# export
class RNNTrainer(Callback):
    def __init__(self, α, β):
        self.α, self.β = α, β

    def after_pred(self):
        # Save the extra outputs for later and only returns the true output.
        self.raw_out, self.out = self.pred[1], self.pred[2]
        self.run.pred = self.pred[0]

    def after_loss(self):
        # AR and TAR
        if self.α != 0.0:
            self.run.loss += self.α * self.out[-1].float().pow(2).mean()
        if self.β != 0.0:
            h = self.raw_out[-1]
            if h.size(1) > 1:
                self.run.loss += self.β * (h[:, 1:] - h[:, :-1]).float().pow(2).mean()

    def begin_epoch(self):
        # Shuffle the texts at the beginning of the epoch
        if hasattr(self.dl.dataset, "batchify"):
            self.dl.dataset.batchify()

Lastly we write a flattened version of the cross entropy loss and the accuracy metric.

In [ ]:
#export
def cross_entropy_flat(input, target):
    bs,sl = target.size()
    return F.cross_entropy(input.view(bs * sl, -1), target.view(bs * sl))

def accuracy_flat(input, target):
    bs,sl = target.size()
    return accuracy(input.view(bs * sl, -1), target.view(bs * sl))

In [ ]:
emb_sz, nh, nl = 300, 300, 2
model = get_language_model(len(vocab), emb_sz, nh, nl, tok_pad, input_p=0.6, output_p=0.4, weight_p=0.5, 
                           embed_p=0.1, hidden_p=0.2)

In [ ]:
cbs = [partial(AvgStatsCallback,accuracy_flat),
       CudaCallback, Recorder,
       partial(GradientClipping, clip=0.1),
       partial(RNNTrainer, α=2., β=1.),
       ProgressCallback]

In [ ]:
learn = Learner(model, data, cross_entropy_flat, lr=5e-3, cb_funcs=cbs, opt_func=adam_opt())

In [ ]:
learn.fit(1)

# Export

In [ ]:
!python notebook2script.py 12a_awd_lstm.ipynb

Converted 12a_awd_lstm.ipynb to exp/nb_12a.py
